In [56]:
import feedparser
import csv
import pandas as pd
import re

Укажите словарь с RSS-лентой, обратите внимание, что не все RSS-ленты имеют доступный для парсинга формат, проверяйте
<br/>Укажите два вектора по которым будет производить поиск, символ '|' означает "или", если нужен только один вектор, в конце другого добавьте '|' - это будет означать, что вектор ищет все слова. Векторы не чувствительны к регистрам
<br/>Укажите два пути для файлов, для всех новостей и для определенных новостей

In [57]:
newsurls = {'Lenta.ru': 'https://lenta.ru/rss/',
'Vesti': 'https://www.vesti.ru/vesti.rss',
'RBC': 'http://static.feed.rbc.ru/rbc/logical/footer/news.rss',
'1CBUH': 'https://buh.ru/rss/?chanel=news',
'Regnum': 'https://regnum.ru/rss/news',
'Lenta': 'https://lenta.ru/rss',
'RT': 'https://www.rt.com/rss/',
'VZGLYAD': 'https://vz.ru/rss.xml',
'RG': 'https://rg.ru/xml/index.xml',
'Interfax': 'https://www.interfax.ru/rss.asp',
'NezGaz': 'https://www.ng.ru/rss/',
'MK': 'https://www.mk.ru/rss/index.xml',
'Klerk.ru': 'https://www.klerk.ru/export/news.rss',
'RT': 'https://www.rt.com/rss/'}

f_all_news = 'allnews23march.csv'
f_certain_news = 'certainnews23march.csv'

vector1 = 'Крым' # в векторах можно указывать сколько угодно слов, но если хочешь, чтобы искало по одному слову,
                 # в конце второго вектора надо ставить | (или)
vector2 = 'Деньги|'

In [58]:
def parseRSS( rss_url ): #функция получает линк на рсс ленту, возвращает распаршенную ленту с помощью feedpaeser
    return feedparser.parse(rss_url)
    
def getHeadlines(rss_url): #функция для получения заголовков новости
    headlines = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        headlines.append(newsitem['title'])
    return headlines

def getDescriptions(rss_url): #функция для получения описания новости
    descriptions = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        descriptions.append(newsitem['description'])
    return descriptions

def getLinks(rss_url): #функция для получения ссылки на источник новости
    links = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        links.append(newsitem['link'])
    return links

def getDates(rss_url): #функция для получения даты публикации новости
    dates = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        dates.append(newsitem['published'])
    return dates

In [59]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем нашии URL и добавляем их в наши пустые списки
for key,url in newsurls.items():
    allheadlines.extend(getHeadlines(url))
    
for key,url in newsurls.items():
    alldescriptions.extend(getDescriptions(url))
    
for key,url in newsurls.items():
    alllinks.extend(getLinks(url))
    
for key,url in newsurls.items():
    alldates.extend(getDates(url))

In [60]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                                                                                         #определенных новостей по таргета,
                                                                                        #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [73]:
write_all_news(f_all_news) #все новости

,Title,Description,Links,Publication Date
0,На Украине пожар на Крымском мосту назвали результатом спецоперации СБУ,Пожар на Крымском мосту через Керченский пролив произошел в результате спецоперации Службы безопасности Украины (СБУ).,https://lenta.ru/news/2022/10/08/spec_sbu/,"Sat, 08 Oct 2022 12:14:00 +0300"
1,В США раскритиковали Байдена за слова о «ядерном Армагеддоне»,"Дочь покойного американского сенатора Джона Маккейна, политический обозреватель Меган Маккейн заявила, что президент США Джо Байден нагнетает страх своими высказываниями о «ядерном Армагеддоне», к которому якобы могут привести действия России. По ее словам, из-за этого жители страны начали паниковать.",https://lenta.ru/news/2022/10/08/bayden_armagedon/,"Sat, 08 Oct 2022 12:08:05 +0300"
2,Названо число туристов в Крыму,NaN,https://lenta.ru/news/2022/10/08/50/,"Sat, 08 Oct 2022 12:08:00 +0300"
3,Приграничный российский регион второй раз за день обстреляли со стороны Украины,Село Тишанка Волоконовского района Белгородской области обстреляли со стороны Украины. Об этом сообщил губернатор приграничного российского региона Вячеслав Гладков в Telegram-канале. Это второй обстрел Белгородской области за сутки. Вооруженные силы Украины (ВСУ) 8 октября также атаковали село Бирюч в Валуйском районе,https://lenta.ru/news/2022/10/08/gladkov/,"Sat, 08 Oct 2022 12:06:00 +0300"
4,Названо число собирающихся покинуть Крым туристов,"Около 4 тысяч туристов должны покинуть Крым в ближайшие 3-4 дня, еще столько же собираются отправиться туда. Число туристов назвал исполняющий обязанности президента Российского союза туриндустрии (РСТ) Илья Уманский. Губернатор Севастополя Михаил Развожаев ранее заявил, что запаса бензина хватит на 40 суток.",https://lenta.ru/news/2022/10/08/34tysyachi/,"Sat, 08 Oct 2022 12:04:00 +0300"
...,...,...,...,...
1107,Региональных мусорных операторов и водителей мусоровозов могут освободить от частичной мобилизации,"Минприроды считает, что это системообразующие предприятия, которые влияют на социальную стабильность регионов.",https://www.klerk.ru/buh/news/537103/,"Fri, 07 Oct 2022 15:43:00 +0300"
1108,Работодатели могут бронировать должность без согласия работника,"Бронь должности делают, чтобы предприятие работало бесперебойно. И поэтому, даже если работник не давал личного согласия на бронь, компания вправе это сделать без разрешения человека.",https://www.klerk.ru/buh/news/537126/,"Fri, 07 Oct 2022 15:24:00 +0300"
1109,При переходе на АУСН с 2023 года будет автоматический отказ от УСН,"С 1 июля 2022 года новый бизнес из 4 регионов может перейти на АУСН. С 1 января 2023 года выбрать этот режим сможет уже действующий бизнес, который сейчас применяет другой режим.",https://www.klerk.ru/buh/news/537133/,"Fri, 07 Oct 2022 15:20:00 +0300"
1110,Спецразрешение для крупногабаритного транспорта можно получить на Госуслугах,Электронные разрешения будут оформлять быстрее бумажных. Срок выдачи разрешения займет от 4 до 9 дней.,https://www.klerk.ru/buh/news/537127/,"Fri, 07 Oct 2022 15:13:00 +0300"


In [74]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) #новости по вектору

,Title,Description,Links,Publication Date
0,На Украине пожар на Крымском мосту назвали результатом спецоперации СБУ,Пожар на Крымском мосту через Керченский пролив произошел в результате спецоперации Службы безопасности Украины (СБУ).,https://lenta.ru/news/2022/10/08/spec_sbu/,"Sat, 08 Oct 2022 12:14:00 +0300"
2,Названо число туристов в Крыму,NaN,https://lenta.ru/news/2022/10/08/50/,"Sat, 08 Oct 2022 12:08:00 +0300"
4,Названо число собирающихся покинуть Крым туристов,"Около 4 тысяч туристов должны покинуть Крым в ближайшие 3-4 дня, еще столько же собираются отправиться туда. Число туристов назвал исполняющий обязанности президента Российского союза туриндустрии (РСТ) Илья Уманский. Губернатор Севастополя Михаил Развожаев ранее заявил, что запаса бензина хватит на 40 суток.",https://lenta.ru/news/2022/10/08/34tysyachi/,"Sat, 08 Oct 2022 12:04:00 +0300"
7,Хуснуллин приедет на место ЧП на Крымском мосту,"Вице-премьер России Марат Хуснуллин, возглавивший правкомиссию по ликвидации последствий ЧП на Крымском мосту, выехал на место происшествия. «В ближайшие часы вылетаю в Крым. Лично осмотрю повреждения моста», — написал вице-премьер. Он пообещал принять решения, которые помогут восстановить работу Крымского моста.",https://lenta.ru/news/2022/10/08/husnullin/,"Sat, 08 Oct 2022 11:54:00 +0300"
8,Налажен сухопутный коридор из Крыма в регионы России через новые территории,"Налажен сухопутный коридор из Крыма в регионы России через новые территории. Такое заявление сделал на фоне перекрытия Крымского моста советник главы Крыма по информационной политике Олег Крючков. Уточняется, что этот коридор пролегает через территории Херсонской, Запорожской областей, а также ДНР.",https://lenta.ru/news/2022/10/08/prolozhen_suhoputni/,"Sat, 08 Oct 2022 11:53:00 +0300"
...,...,...,...,...
1084,Кондратьев: транспортного коллапса из-за ЧП на Крымском мосту нет,"Вениамин Кондратьев в своем телеграм-канале написал, что транспортного коллапса из-за ЧП на Крымском мосту нет",https://kuban.mk.ru/social/2022/10/08/kondratev-transportnogo-kollapsa-izza-chp-na-krymskom-mostu-net.html,"Sat, 08 Oct 2022 11:18:55 +0300"
1086,Теракт на Крымском мосту вызвал неудобные вопросы: почему не досматривали машины,Репортаж очевидца,https://www.mk.ru/social/2022/10/08/terakt-na-krymskom-mostu-zadal-voprosy-pochemu-ne-dosmatrivali-mashiny.html,"Sat, 08 Oct 2022 11:16:35 +0300"
1087,Минобороны Украины прокомментировало инцидент на Крымском мосту,В Официальном аккаунте Министерства обороны Украины появился издевательский комментарий относительно ЧП на Крымском мосту,https://www.mk.ru/politics/2022/10/08/minoborony-ukrainy-prokommentirovalo-incident-na-krymskom-mostu.html,"Sat, 08 Oct 2022 11:15:59 +0300"
1090,Петербуржцы не смогут поехать в Крым на поезде в ближайшее время,"Петербуржцы не смогут поехать в Крым на поезде в ближайшее время. Как сообщили в пресс-службе РЖД, продажа билетов на поезда в сообщении с полуостровом приостановлена.",https://spb.mk.ru/social/2022/10/08/peterburzhcy-ne-smogut-poekhat-v-krym-na-poezde-v-blizhayshee-vremya.html,"Sat, 08 Oct 2022 11:10:39 +0300"


In [75]:
#выделяю все новости в отдельный датафрейм для лемматизации
news_df = pd.DataFrame(write_all_news(f_all_news))

In [64]:
news_df

,Title,Description,Links,Publication Date
0,На Украине пожар на Крымском мосту назвали результатом спецоперации СБУ,Пожар на Крымском мосту через Керченский пролив произошел в результате спецоперации Службы безопасности Украины (СБУ).,https://lenta.ru/news/2022/10/08/spec_sbu/,"Sat, 08 Oct 2022 12:14:00 +0300"
1,В США раскритиковали Байдена за слова о «ядерном Армагеддоне»,"Дочь покойного американского сенатора Джона Маккейна, политический обозреватель Меган Маккейн заявила, что президент США Джо Байден нагнетает страх своими высказываниями о «ядерном Армагеддоне», к которому якобы могут привести действия России. По ее словам, из-за этого жители страны начали паниковать.",https://lenta.ru/news/2022/10/08/bayden_armagedon/,"Sat, 08 Oct 2022 12:08:05 +0300"
2,Названо число туристов в Крыму,NaN,https://lenta.ru/news/2022/10/08/50/,"Sat, 08 Oct 2022 12:08:00 +0300"
3,Приграничный российский регион второй раз за день обстреляли со стороны Украины,Село Тишанка Волоконовского района Белгородской области обстреляли со стороны Украины. Об этом сообщил губернатор приграничного российского региона Вячеслав Гладков в Telegram-канале. Это второй обстрел Белгородской области за сутки. Вооруженные силы Украины (ВСУ) 8 октября также атаковали село Бирюч в Валуйском районе,https://lenta.ru/news/2022/10/08/gladkov/,"Sat, 08 Oct 2022 12:06:00 +0300"
4,Названо число собирающихся покинуть Крым туристов,"Около 4 тысяч туристов должны покинуть Крым в ближайшие 3-4 дня, еще столько же собираются отправиться туда. Число туристов назвал исполняющий обязанности президента Российского союза туриндустрии (РСТ) Илья Уманский. Губернатор Севастополя Михаил Развожаев ранее заявил, что запаса бензина хватит на 40 суток.",https://lenta.ru/news/2022/10/08/34tysyachi/,"Sat, 08 Oct 2022 12:04:00 +0300"
...,...,...,...,...
1107,Региональных мусорных операторов и водителей мусоровозов могут освободить от частичной мобилизации,"Минприроды считает, что это системообразующие предприятия, которые влияют на социальную стабильность регионов.",https://www.klerk.ru/buh/news/537103/,"Fri, 07 Oct 2022 15:43:00 +0300"
1108,Работодатели могут бронировать должность без согласия работника,"Бронь должности делают, чтобы предприятие работало бесперебойно. И поэтому, даже если работник не давал личного согласия на бронь, компания вправе это сделать без разрешения человека.",https://www.klerk.ru/buh/news/537126/,"Fri, 07 Oct 2022 15:24:00 +0300"
1109,При переходе на АУСН с 2023 года будет автоматический отказ от УСН,"С 1 июля 2022 года новый бизнес из 4 регионов может перейти на АУСН. С 1 января 2023 года выбрать этот режим сможет уже действующий бизнес, который сейчас применяет другой режим.",https://www.klerk.ru/buh/news/537133/,"Fri, 07 Oct 2022 15:20:00 +0300"
1110,Спецразрешение для крупногабаритного транспорта можно получить на Госуслугах,Электронные разрешения будут оформлять быстрее бумажных. Срок выдачи разрешения займет от 4 до 9 дней.,https://www.klerk.ru/buh/news/537127/,"Fri, 07 Oct 2022 15:13:00 +0300"


**Попытки лемматизировать:**

In [65]:
from pymystem3 import Mystem

In [66]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [67]:
data = ["programmers program with programming languages", "my code is working so there must be a bug in the interpreter"]
df = pd.DataFrame(data, columns = ['unstemmed'])
df['unstemmed'] = df['unstemmed'].str.split()
pd.set_option('display.max_colwidth', -1)
df 

C:\Users\mhadj\AppData\Local\Temp\ipykernel_12024\3145216783.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,unstemmed
0,"[programmers, program, with, programming, languages]"
1,"[my, code, is, working, so, there, must, be, a, bug, in, the, interpreter]"


In [76]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return (','.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]))

df = pd.DataFrame(['это было круто', 'ей нравятся эти книги нравится нравиться нравится нравится', 'wow this is great'], columns=['text'])
df['text_lemmatized'] = df.text.apply(lemmatize_text)
df

,text,text_lemmatized
0,это было круто,"это,было,круто"
1,ей нравятся эти книги нравится нравиться нравится нравится,"ей,нравятся,эти,книги,нравится,нравиться,нравится,нравится"
2,wow this is great,"wow,this,is,great"


In [77]:
import json
from pymystem3 import Mystem

for i in df['text_lemmatized']:
    m = Mystem()
    lemmas = m.lemmatize(text)
    text = i
    print("lemmas:", ''.join(lemmas))

lemmas: wow,this,is,great

lemmas: это,быть,круто

lemmas: она,нравиться,этот,книга,нравиться,нравиться,нравиться,нравиться

